# Introduction

In this notebook, we implement [YOLOv4](https://arxiv.org/pdf/2004.10934.pdf) for training on your own dataset in PyTorch.

We also recommend reading our blog post on [Training YOLOv4 on custom data](https://blog.roboflow.ai/training-yolov4-on-a-custom-dataset/) side by side.

We will take the following steps to implement YOLOv4 on our custom data:
* Set up YOLOv4 environment
* Download Custom Dataset via Roboflow
* Train Custom YOLOv4 detector
* Reload Custom YOLOv4 detector weights for inference

When you are done you will have a custom detector that you can use. It will make inference like this:

#### ![Chesse Image](https://i.imgur.com/cGOxffX.png)

### **Reach out for support**

If you run into any hurdles on your own data set or just want to share some cool results in your own domain, [reach out!](https://roboflow.ai/contact) 


#### ![Roboflow Workmark](https://i.imgur.com/WHFqYSJ.png)

# Set up YOLOv4 Environment

In [ ]:
!git clone https://github.com/roboflow-ai/pytorch-YOLOv4.git

Cloning into 'pytorch-YOLOv4'...
remote: Enumerating objects: 392, done.
remote: Total 392 (delta 0), reused 0 (delta 0), pack-reused 392
Receiving objects: 100% (392/392), 914.64 KiB | 20.33 MiB/s, done.
Resolving deltas: 100% (223/223), done.


In [ ]:
!pip install pyopenssl ndg-httpsclient pyasn1

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 2.6MB 13.4MB/s 


In [ ]:
%cd /content/pytorch-YOLOv4
!pip install -r requirements.txt

/content/pytorch-YOLOv4
     |████████████████████████████████| 20.2MB 64.6MB/s 
     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 12.6MB 237kB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 2.1MB 36.7MB/s 
     |████████████████████████████████| 317kB 36.3MB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.3 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
  

In [ ]:
# download yolov4 weights that have already been converted to Python
!gdown https://drive.google.com/uc?id=1fcbR0bWzYfIEdLJPzOsn4R5mlvR6IQyA

# Download Custom Dataset

## Export Your Dataset from Roboflow

Roboflow enables you to export your dataset in any format you need - including for this notebook.

Create a [free account](https://app.roboflow.ai). Upload your private dataset. Generate a version (applying any preprocessing and augmentations you desire). Create an export. Select **YOLOv4 PyTorch** as the export format. Click **"Show Download code"**, copy your link, and paste it in the next cell. Magic.

In [ ]:
%%time
# REPLACE this link with your Roboflow dataset (export as YOLOv4 PyTorch format)
!curl -L "https://app.roboflow.com/ds/IoeDPeuY0z?key=BbnbigmG5a" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0    987      0 --:--:-- --:--:-- --:--:--   986
100 74.0M  100 74.0M    0     0  38.1M      0  0:00:01  0:00:01 --:--:--  188M
Archive:  roboflow.zip
 extracting: train/bcb2d8_jpg.rf.00c850d52486d81f3f1c4dcb73e21446.jpg  
 extracting: train/0968cc_jpg.rf.02be0fbd26a0344551ef65a39f716ac7.jpg  
 extracting: train/b1fb5c_jpg.rf.02c0f366554f6e79692d0ed3dae29e26.jpg  
 extracting: train/6f9fed_jpg.rf.009645d9d3938e91b901bfc518c8ce33.jpg  
 extracting: train/8ad377_jpg.rf.0370b7d64c906eda65f455d436ef5655.jpg  
 extracting: train/76ab88_jpg.rf.027c1fcb73373ac22897cfce8d2b76bf.jpg  
 extracting: train/bf6aaf_jpg.rf.01a5a4f34b4580fc04cf717252d005ef.jpg  
 extracting: train/ed6f88_jpg.rf.0128943f494bbbe61e28282c67d09500.jpg  
 extracting: train/2e5f8d_jpg.rf.03c33cdae943f34df831016985fb9027.jpg  
 extracting: 

In [ ]:
%%time
%cp train/_annotations.txt train/train.txt
%cp train/_annotations.txt train.txt
%cp valid/_annotations.txt data/val.txt
%cp valid/*.jpg train/

CPU times: user 8.26 ms, sys: 19.8 ms, total: 28 ms
Wall time: 441 ms


In [ ]:
%%time
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_classes.txt')

CPU times: user 482 µs, sys: 129 µs, total: 611 µs
Wall time: 574 µs


In [ ]:
%%time
print(num_classes)

12
CPU times: user 657 µs, sys: 0 ns, total: 657 µs
Wall time: 564 µs


# Train Custom Detector

In [ ]:
%%time
#start training
#-b batch size (you should keep this low (2-4) for training to work properly)
#-s number of subdivisions in the batch, this was more relevant for the darknet framework
#-l learning rate
#-g direct training to the GPU device
#pretrained invoke the pretrained weights that we downloaded above
#classes - number of classes
#dir - where the training data is 
#epoch - how long to train for
!python resume_train.py -b 2 -s 1 -l 0.001 -g 0 -pretrained /content/gdrive/My\ Drive/Vivadata/final_project/ai_meter_reading/models_trained/params_to_load/20200929_130852/Yolov4_epoch99.pth -classes {num_classes} -dir ./train -epochs 100

Epoch 56/100:  99%|▉| 1408/1418 [05:06<00:02,  4.92020-09-30 09:53:42,459 resume_train.py[line:360] INFO: Train step_39700: loss : 75.97970581054688,loss xy : 51.75459671020508,loss wh : 2.003539562225342,loss obj : 10.841917991638184，loss cls : 11.379651069641113,loss l2 : 11.152591705322266,lr : 0.0001
Epoch 56/100: 100%|█| 1418/1418 [05:08<00:00,  4.5
Epoch 57/100:  98%|▉| 1390/1418 [05:02<00:05,  4.72020-09-30 09:58:46,810 resume_train.py[line:360] INFO: Train step_40400: loss : 29.322702407836914,loss xy : 28.844724655151367,loss wh : 0.22456571459770203,loss obj : 0.18583843111991882，loss cls : 0.0675729438662529,loss l2 : 0.7110680937767029,lr : 0.0001
Epoch 57/100: 100%|█| 1418/1418 [05:08<00:00,  4.6
Epoch 58/100: 100%|▉| 1412/1418 [05:07<00:01,  4.92020-09-30 10:04:00,087 resume_train.py[line:360] INFO: Train step_41120: loss : 68.17391967773438,loss xy : 49.49085998535156,loss wh : 1.5696296691894531,loss obj : 6.949719429016113，loss cls : 10.163713455200195,loss l2 : 8.4642

# Load Trained Weights for Custom Detection

You can also use this to load previously saved weights!

In [ ]:
%cd /content/gdrive/My\ Drive/Vivadata/final_project/ai_meter_reading/models_trained/yolo_v4_20200930_134059/pytorch-YOLOv4

/content/gdrive/My Drive/Vivadata/final_project/ai_meter_reading/models_trained/yolo_v4_20200930_134059/pytorch-YOLOv4


In [ ]:
!ls checkpoints

Yolov4_criterion_epoch100.pth  Yolov4_optimizer_epoch100.pth
Yolov4_criterion_epoch1.pth    Yolov4_optimizer_epoch1.pth
Yolov4_criterion_epoch92.pth   Yolov4_optimizer_epoch92.pth
Yolov4_criterion_epoch93.pth   Yolov4_optimizer_epoch93.pth
Yolov4_criterion_epoch94.pth   Yolov4_optimizer_epoch94.pth
Yolov4_criterion_epoch95.pth   Yolov4_optimizer_epoch95.pth
Yolov4_criterion_epoch96.pth   Yolov4_optimizer_epoch96.pth
Yolov4_criterion_epoch97.pth   Yolov4_optimizer_epoch97.pth
Yolov4_criterion_epoch98.pth   Yolov4_optimizer_epoch98.pth
Yolov4_criterion_epoch99.pth   Yolov4_optimizer_epoch99.pth
Yolov4_epoch100.pth	       Yolov4_scheduler_epoch100.pth
Yolov4_epoch1.pth	       Yolov4_scheduler_epoch1.pth
Yolov4_epoch92.pth	       Yolov4_scheduler_epoch92.pth
Yolov4_epoch93.pth	       Yolov4_scheduler_epoch93.pth
Yolov4_epoch94.pth	       Yolov4_scheduler_epoch94.pth
Yolov4_epoch95.pth	       Yolov4_scheduler_epoch95.pth
Yolov4_epoch96.pth	       Yolov4_scheduler_epoch96.pth
Yolov4_epoch97.

In [ ]:
# download the test set
!curl -L "https://app.roboflow.com/ds/FNXUko6fnC?key=hAne2aIkcf" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cp train/_classes.txt test/_classes.txt

In [ ]:
%%time
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_classes.txt')

CPU times: user 1.35 ms, sys: 23 µs, total: 1.38 ms
Wall time: 11.2 s


In [ ]:
%%time
print(num_classes)

12
CPU times: user 59 µs, sys: 0 ns, total: 59 µs
Wall time: 62 µs


In [ ]:
#choose random test image
import os
test_images = [f for f in os.listdir('test') if f.endswith('.jpg')]
import random
img_path = "test/" + random.choice(test_images);

In [ ]:
img_path, len(test_images)

('test/fmigjg_jpg.rf.3c4c886678903833b7b01dfae1c4a157.jpg', 416)

In [ ]:
%%time
##change the epoch here to the one you would like to use for inference
# !python models.py {num_classes} checkpoints/Yolov4_epoch50.pth {img_path} test/_classes.txt

In [ ]:
#visualize inference
# from IPython.display import Image
# Image('predictions.jpg')

In [ ]:
%%time
# make prediction for each file in the test folder
i = 1
for file in test_images:
  print(i, ': ', file)
  img_path = 'test/' + file
  !python models.py {num_classes} checkpoints/Yolov4_epoch92.pth {img_path} test/_classes.txt
  i += 1
  print('------------------------------------------------------------------------------------------------')

1 :  ekgpdl_jpg.rf.102720674227fb03df312188604d723a.jpg
3: 0.999954
1: 0.951361
3: 0.999999
counter: 0.999998
liters: 1.000000
3: 0.672605
save plot results to test_pred_20200930_134059_epoch92/ekgpdl_jpg.rf.102720674227fb03df312188604d723a_pred_20200930_134059_epoch92.jpg
------------------------------------------------------------------------------------------------
2 :  koniaa_jpg.rf.504c790cdc922866db66d1c1f4d4aefa.jpg
0: 1.000000
7: 0.999873
0: 0.999999
6: 0.977126
counter: 0.999994
8: 0.772390
liters: 0.999999
save plot results to test_pred_20200930_134059_epoch92/koniaa_jpg.rf.504c790cdc922866db66d1c1f4d4aefa_pred_20200930_134059_epoch92.jpg
------------------------------------------------------------------------------------------------
3 :  bmdfom_jpg.rf.6b548562c07f22c5b12516f6c77d0cc0.jpg
liters: 1.000000
1: 0.999998
0: 1.000000
5: 0.996226
1: 0.999528
3: 1.000000
counter: 1.000000
save plot results to test_pred_20200930_134059_epoch92/bmdfom_jpg.rf.6b548562c07f22c5b12516f6c7